In [2]:
import os
import pandas as pd
import numpy as np
import re
import sys
from collections import Counter
import itertools
scripts_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts'))
sys.path.insert(0, scripts_dir)
import data_processing as data_pr



In [3]:
#define paths and names
data_paths = data_pr.get_datapaths('HousePrices')
data_in, data_info = data_pr.get_data(data_paths, print_output=True, split='train')

data info: 
MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	       Agriculture
       C (all)	       Commercial
       FV	       Floating Village Residential
       I	       Industrial
       RH	       Residential High Density
       RL	       Residential L

In [3]:
x = re.findall('^\w+:.+?\n\s*\n', data_info, flags=re.MULTILINE)
categs = [heading.split(':')[0] for heading in x]
descripts = re.split('^\w+:.+?\n\s*\n', data_info, flags=re.MULTILINE)[1:]
options = [re.findall('^\s+([\w|\.|\&| |\(\|\)]+)\t', des, flags=re.MULTILINE) for des in descripts]
print('number of descriptions: ', len(descripts))
print('numbers of categories: ', len(categs))
options_dict = {cat:opt for cat, opt in zip(categs, options)}
for k, v in options_dict.items():
    print(f"{k}: {v}")


number of descriptions:  79
numbers of categories:  79
MSSubClass: ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190']
MSZoning: ['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']
LotFrontage: []
LotArea: []
Street: ['Grvl', 'Pave', ' ']
Alley: ['Grvl', 'Pave', 'NA']
LotShape: ['Reg', 'IR1', 'IR2', 'IR3']
LandContour: ['Lvl', 'Bnk', 'HLS', 'Low']
Utilities: ['AllPub', 'NoSewr', 'NoSeWa', 'ELO']
LotConfig: ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3']
LandSlope: ['Gtl', 'Mod', 'Sev']
Neighborhood: ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker']
Condition1: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe']
Condition2: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe']
Bldg

In [4]:
#different methods of transforming data
scalar_inputs = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',\
                 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageCars', 'GarageArea',\
                 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'YrSold', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',\
                    'KitchenAbvGr', 'Fireplaces']
to_scalar = ['LotShape', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'CentralAir', 'Functional',\
             'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'KitchenQual', 'FireplaceQu', 'HeatingQC']
one_hot = ['MSSubClass', 'MSZoning', 'Street', 'LotConfig', 'Neighborhood', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'Foundation',\
            'SaleType', 'SaleCondition']
one_hot_plus = {'Alley': {'NA': 0}, 'LandContour': {'Lvl': 0}, 'MasVnrType': {'None': 0, 'NA': 'dist', 'weight': 'MasVnrArea'}, 'GarageType':{'NA': 0},\
                 'MiscFeature': {'NA': 0, 'weight': 'MiscVal'}, 'Heating': {'weight': 'HeatingQC'}}
case_by_case = ['Utilities', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'BsmtFinType1', 'BsmtFinType2',\
               'Electrical']
#post processing
#delete TotalBsmtSF
#do to_scalar transformation and replace cols in data_in
#transform MoSold and add to year
#transform one hot, one hot plus and case to case entries. Store in a dictionary.
#Change NA in Electrical to dist of the other entries. Remove NA
#combine the transformed entries in the dict back into the original dataframe. Check for repeats
#take mean and var of each feature and scale and renormalize

In [5]:
#get types in options_dict to match the types in data_in
for k in data_in.keys():
    if k in options_dict:
        if k in one_hot+list(one_hot_plus.keys()):
            if type(data_in[k][0])!=str:
                data_in[k] = data_in[k].apply(str)
        #gather the unique elements in data_in[k], the set of all values of the variable with heading k
        actual_opts = list(set(data_in[k].tolist()))
        actual_type = type(actual_opts[0])
        #if the type of the variable is something other than str, need to convert options_dict variable options to the correct type
        if options_dict[k] == []:  #this represents a free numeric entry
            options_dict[k] = {'variable_type': actual_type}  #what type is this entry in data_in to start?
            #keep working here
        else:
            #if the type of a column in data_in is something other than a string, 
            #the options of the corresponding key in options_dict will have to be converted 
            if actual_type != str: 
                options_dict[k] = [actual_type(item) for item in options_dict[k]]

            #check if there exist any items in data_in that do not appear in opitons_dict and raise an error if so
            check_options = [item in options_dict[k] for item in actual_opts]
            if not all(check_options):
                print(check_options)
                false_idx = -1
                num_false = check_options.count(False)
                for n in range(num_false):
                    false_idx = check_options.index(False, false_idx+1)
                    print(f'ERROR: value {actual_opts[false_idx]} for key {k} does not exist in options_dict')           
    #if this key does not exist in options_dict and it is not a special key, then raise an error
    elif k !='SalePrice' and k!= 'Id':
        print(f'ERROR: key {k} in the data is not in options_dict')

for k, v in options_dict.items():
    print(f"{k}: {v}")     

MSSubClass: ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190']
MSZoning: ['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']
LotFrontage: {'variable_type': <class 'str'>}
LotArea: {'variable_type': <class 'int'>}
Street: ['Grvl', 'Pave', ' ']
Alley: ['Grvl', 'Pave', 'NA']
LotShape: ['Reg', 'IR1', 'IR2', 'IR3']
LandContour: ['Lvl', 'Bnk', 'HLS', 'Low']
Utilities: ['AllPub', 'NoSewr', 'NoSeWa', 'ELO']
LotConfig: ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3']
LandSlope: ['Gtl', 'Mod', 'Sev']
Neighborhood: ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker']
Condition1: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe']
Condition2: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe']

In [6]:
float_type = float
for k, v in data_in.items():
    all_numeric=False
    free_var=False
    if k in options_dict:
        #print(k)
        #check if all options are numeric 
        if type(options_dict[k])==list: #if options are explicitly listed and they are strings then check if they are strings of numeric values
            if type(options_dict[k][0])==str:
                numeric = [item.replace('.', '').replace('-', '').replace(' ', '').isnumeric() for item in options_dict[k]] #remove special characters
                all_numeric = all(numeric)
        elif type(options_dict[k])==dict: #if options are described by a dictionary then the the data type is a free form number
            if options_dict[k]['variable_type']==str: #if that number is currently in the form of a string then it must be converted
                all_numeric=True
                free_var=True
        if all_numeric and k != 'MSSubClass':
            #print(options_dict[k])

            #before converting, replace 'NA' with -1
            data_in[k] = data_in[k].replace('NA', -1)
            data_in[k] = data_in[k].astype(float_type) #convert data
            #print('type: ', type(data_in[k].loc[1]))

            #convert options dict as well
            if free_var:
                options_dict[k]['variable_type']=float_type
            else:
                options_dict[k] = [float_type(item) for item in options_dict[k]]

for k, v in options_dict.items():
    print(f"{k}: {v}")

MSSubClass: ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190']
MSZoning: ['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']
LotFrontage: {'variable_type': <class 'float'>}
LotArea: {'variable_type': <class 'int'>}
Street: ['Grvl', 'Pave', ' ']
Alley: ['Grvl', 'Pave', 'NA']
LotShape: ['Reg', 'IR1', 'IR2', 'IR3']
LandContour: ['Lvl', 'Bnk', 'HLS', 'Low']
Utilities: ['AllPub', 'NoSewr', 'NoSeWa', 'ELO']
LotConfig: ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3']
LandSlope: ['Gtl', 'Mod', 'Sev']
Neighborhood: ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker']
Condition1: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe']
Condition2: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe

In [7]:
#count keys to make sure they add up to the right number
full_list = one_hot + scalar_inputs +  to_scalar + case_by_case + list(one_hot_plus.keys()) + ['MasVnrArea', 'MiscVal']
for item in full_list:
    if item not in categs:
        print(item)
his = Counter(full_list)
for k,v in his.items():
    if v>1:
        print(f'key {k} has more than one occurance')


In [8]:
def add_to_series(old_series, pos, new_label, val):
    new_index  = old_series.index.insert(pos, new_label)
    # insert into the underlying array of values
    new_values = np.insert(old_series.values, pos, val)
    new_series = pd.Series(new_values, index=new_index)
    return new_series

def input_dist(df_new, key, dist=None):
    if dist is None:
        dist = df_new[df_new[key]!=1.].mean() #find distribution of different values excluding unknowns
    else:
        dist = add_to_series(dist, df_new.columns.get_loc(key), key, 0) #add an entry to dist for key
    df_new[df_new[key]==1] = dist #Give the unknowns the distribution of inputs
    #drop unknown column from df_new and dist
    df_new = df_new.drop(columns=[key])
    dist = dist.drop(index=key)
    return df_new, dist

def to_one_hot(data_in, categ, dtype=float):
    num_data = len(data_in)
    one_col = data_in[categ] #extract the column we want to transform
    actual_opts = list(set(one_col.tolist())) #extract possible values entries in that column have
    #print(actual_opts)
    to_add = []
    for item in options_dict[categ]:
        #add missing entries
        if item not in actual_opts:
            one_col.loc[len(one_col)] = item  # adding a row
    df1 = pd.get_dummies(one_col, dtype=dtype) #transform to one hot encoding
    df1 = df1.loc[:num_data-1] #remove extra rows
    return df1

def to_one_hot_plus(data_in, categ, conv_dict, dist=None):
    df_new = to_one_hot(data_in, categ) #transform to one hot encoding
    #consider special cases
    for k2, val in conv_dict.items():
        if val==0:
            df_new = df_new.drop(columns=[k2])
        elif val=='dist':
            df_new, dist = input_dist(df_new, k2, dist=dist)
        elif k2=='weight':
            df_new = df_new.mul(data_in[val], axis=0)
    if dist is not None:
        dist = dist[dist.index.isin(df_new.columns)]

    return df_new, dist

def create_mappings(to_scalar, options_dict):
    mappings = {}
    for k in to_scalar:
        num_opts = len(options_dict[k])
        
        # define mapping in descending order from 1 to 0.
        mappings[k] = {opt: 1-i/(num_opts-1) for i, opt in enumerate(options_dict[k])}
    return mappings

def cat_to_vec(init_df, mapping, new_cols=None, cat=None):
    if cat is None:
        cat = init_df.columns[0]
    vec = init_df[cat].map(mapping)

    # turn list‐values into real columns
    vec_df = pd.DataFrame(vec.tolist(),
                    index=init_df.index,
                    columns=new_cols)
    return vec_df

In [9]:
print(data_in)

        Id MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0      635         90       RL         64.0     6979   Pave    NA      Reg   
1       18         90       RL         72.0    10791   Pave    NA      Reg   
2      155         30       RM         84.0    11340   Pave    NA      Reg   
3      971         50       RL         60.0    10800   Pave    NA      Reg   
4     1415         50       RL         64.0    13053   Pave  Pave      Reg   
...    ...        ...      ...          ...      ...    ...   ...      ...   
1089  1028         20       RL         71.0     9520   Pave    NA      IR1   
1090  1419         20       RL         71.0     9204   Pave    NA      Reg   
1091   810         75       RM         90.0     8100   Pave    NA      Reg   
1092  1208         20       RL         70.0     9135   Pave    NA      Reg   
1093   903         60       RL         63.0     7875   Pave    NA      Reg   

     LandContour Utilities  ... PoolArea PoolQC  Fence MiscFeat

In [10]:
#transform columns in the to_scalar category
mappings = create_mappings(to_scalar, options_dict)
for k in to_scalar:
    data_in[k] = data_in[k].map(mappings[k])

In [11]:
data_in = data_in.drop(columns='TotalBsmtSF')
data_in['YrSold'] = data_in['YrSold'] + data_in['MoSold']/12 - 1/24


In [12]:
data_in = data_in.drop(columns='MoSold')
data_in.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [13]:
for key in one_hot + list(one_hot_plus.keys()):
    if type(data_in[key][0])!=str:
        print(key)
        data_in[key] = data_in[key].apply(str)
        print(type(data_in[key][0]))


In [14]:
# transform those in the one_hot_plus category
dfs={}
dists = {}
for k, val in one_hot_plus.items():
    df_new, dist = to_one_hot_plus(data_in, k, val)
    if dist is not None:
        dists[k] = dist
    dfs[k] = df_new


In [15]:
# transform those in the one_hot category
for categ in one_hot:
    df1 = to_one_hot(data_in, categ)
    dfs[categ] = df1
    #df = pd.concat([df, df1.drop(list(np.arange(len(data_in), len(df1))))], axis=1)



In [16]:
#define mappings for case by case features
mappings = {'Utilities': [{'AllPub':[1, 1, 1], 'NoSewr':[0, 1, 1], 'NoSeWa':[0, 0, 1], 'ELO':[0, 0, 0]}, ['Sew', 'Wat', 'Gas']],
           'Condition1': [{'Artery':[1, 0, 0, 0], 'Feedr':[0.5, 0, 0, 0], 'Norm': [0, 0, 0, 0], 'RRNn': [0, 0.5, 0, 0],
                           'RRAn':[0, 1, 0, 0], 'PosN':[0, 0, 0, .5], 'PosA': [0, 0, 0, 1], 'RRNe':[0, 0, 0.5, 0], 'RRAe': [0, 0, 1, 0]},
                           ['Road', 'NS_rail', 'EW_rail', 'Positive']],#add to Condition2
            'Condition2': 'Condition1',
            'BldgType': [{'1Fam':[0, 0], '2fmCon':[1, 0], 'Duplex': [0.5, 0], 'TwnhsE':[0, 0.5], 'TwnhsI': [0, 1], 'Twnhs':[0, 1]},
                           ['2fam', 'Twnhs']],
            'HouseStyle': [{'1Story': [0, 0, 0], '1.5Fin': [0.4, 0, 0], '1.5Unf': [0.2, 0, 0], '2Story': [0.6, 0, 0], '2.5Fin':[1, 0, 0],
                            '2.5Unf':[0.8, 0, 0], 'SFoyer': [0, 1, 0], 'SLvl': [0, 0, 1]}, ['Stories', 'SFoyer', 'SLvl']],
            'BsmtFinType1': [{'NA': [0], 'Unf': [0], 'LwQ':[0.2], 'Rec':[0.4], 'BLQ':[0.6], 'ALQ':[0.8], 'GLQ':[1]},
                             ['BsmtFiQual1']],
       	    'BsmtFinType2': [{'NA': [0], 'Unf': [0], 'LwQ':[0.2], 'Rec':[0.4], 'BLQ':[0.6], 'ALQ':[0.8], 'GLQ':[1]},
                             ['BsmtFiQual2']],
            'Electrical': [{'SBrkr':[1, 0, 0], 'FuseA':[2/3, 0, 0], 'FuseF':[1/3, 0, 0], 'FuseP':[0, 0, 0], 'Mix':[0, 1, 0], 'NA':[0, 0, 1]},
                           ['BreakWire_rate', 'Mixed', 'NA']]
                           }

In [17]:
k = 'Utilities'
print(data_in[k])
new_df = cat_to_vec(data_in, mappings[k][0], new_cols=mappings[k][1], cat=k)
print(new_df)

0       AllPub
1       AllPub
2       AllPub
3       AllPub
4       AllPub
         ...  
1089    AllPub
1090    AllPub
1091    AllPub
1092    AllPub
1093    AllPub
Name: Utilities, Length: 1094, dtype: object
      Sew  Wat  Gas
0       1    1    1
1       1    1    1
2       1    1    1
3       1    1    1
4       1    1    1
...   ...  ...  ...
1089    1    1    1
1090    1    1    1
1091    1    1    1
1092    1    1    1
1093    1    1    1

[1094 rows x 3 columns]


In [18]:
for k, v in mappings.items():
    if type(v)==str:
        v = mappings[mappings[k]]
    dfs[k] = cat_to_vec(data_in, v[0], new_cols=v[1], cat=k)

In [19]:
np.where(data_in['Electrical']=='NA')

(array([867]),)

In [20]:
df_new, dist = input_dist(dfs['Electrical'], 'NA', dist=None)
dists['Electrical'] = dist
dfs['Electrical'] = df_new


/var/folders/7v/xhq0kcms0ln_ymwzlkm498c40000gn/T/ipykernel_3424/3632145224.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0009149130832570906' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_new[df_new[key]==1] = dist #Give the unknowns the distribution of inputs


In [21]:
data_in = data_in.drop(columns=list(dfs.keys()))

In [22]:
data_in.columns

Index(['Id', 'LotFrontage', 'LotArea', 'LotShape', 'LandSlope', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'HeatingQC', 'CentralAir', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscVal', 'YrSold', 'SalePrice'],
      dtype='object')

In [23]:
cols_dict = {k: list(v.columns) for k, v in dfs.items()}
all_keys = list(itertools.chain(*[v for v in cols_dict.values()]))
freq = Counter(all_keys)
repeat_cols=[]
for k, v in freq.items():
    if v>=2:
        print('column ', k, ' has repeats')
        repeat_cols.append(k)
repeat_keys = []
s_rcol = set(repeat_cols)
for k, v in cols_dict.items():
    if any(item in s_rcol for item in v):
        repeat_keys.append(k)

    


column  Grvl  has repeats
column  Pave  has repeats
column  BrkFace  has repeats
column  CBlock  has repeats
column  Stone  has repeats
column  Shed  has repeats
column  AsbShng  has repeats
column  AsphShn  has repeats
column  HdBoard  has repeats
column  ImStucc  has repeats
column  MetalSd  has repeats
column  Other  has repeats
column  Plywood  has repeats
column  PreCast  has repeats
column  Stucco  has repeats
column  VinylSd  has repeats
column  Wd Sdng  has repeats
column  Road  has repeats
column  NS_rail  has repeats
column  EW_rail  has repeats
column  Positive  has repeats


In [24]:
print(repeat_keys)

['Alley', 'MasVnrType', 'MiscFeature', 'Street', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'Foundation', 'Condition1', 'Condition2']


In [25]:
for k in dfs.keys():
    #print(f"type of {k}: ", type(k))
    #print(f"type of cols of {k}", type(dfs[k].columns[0]))
    if k in repeat_keys:
        pref = k
    else:
        pref = k[:3]
    rename_dict = {name: pref + '_' + name for name in cols_dict[k]}
    dfs[k] = dfs[k].rename(columns=rename_dict)

print(dfs['Electrical'])

      Ele_BreakWire_rate  Ele_Mixed
0               1.000000        0.0
1               1.000000        0.0
2               0.666667        0.0
3               0.666667        0.0
4               1.000000        0.0
...                  ...        ...
1089            1.000000        0.0
1090            1.000000        0.0
1091            0.666667        0.0
1092            1.000000        0.0
1093            1.000000        0.0

[1094 rows x 2 columns]


In [ ]:
for v in dfs.values():
    data_in = pd.concat([data_in, v], axis=1)

In [27]:
print(list(data_in.columns))

['Id', 'LotFrontage', 'LotArea', 'LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'HeatingQC', 'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'YrSold', 'SalePrice', 'Alley_Grvl', 'Alley_Pave', 'Lan_Bnk', 'Lan_HLS', 'Lan_Low', 'MasVnrType_BrkCmn', 'MasVnrType_BrkFace', 'MasVnrType_CBlock', 'MasVnrType_Stone', 'Gar_2Types', 'Gar_Attchd', 'Gar_Basment', 'Gar_BuiltIn', 'Gar_CarPort', 'Gar_Detchd', 'MiscFeature_Elev', 'MiscFeature_Gar2', 'MiscFea

In [28]:
print(data_in)

        Id  LotFrontage  LotArea  LotShape  LandSlope  OverallQual  \
0      635         64.0     6979  1.000000        1.0            6   
1       18         72.0    10791  1.000000        1.0            4   
2      155         84.0    11340  1.000000        1.0            6   
3      971         60.0    10800  1.000000        1.0            4   
4     1415         64.0    13053  1.000000        1.0            6   
...    ...          ...      ...       ...        ...          ...   
1089  1028         71.0     9520  0.666667        1.0            8   
1090  1419         71.0     9204  1.000000        1.0            5   
1091   810         90.0     8100  1.000000        1.0            5   
1092  1208         70.0     9135  1.000000        1.0            6   
1093   903         63.0     7875  1.000000        1.0            7   

      OverallCond  YearBuilt  YearRemodAdd  MasVnrArea  ...  \
0               5       1980          1980         0.0  ...   
1               5       1967     

In [27]:
#check number of one_hot encoding columns
total = 0
for item in one_hot:
    total += len(options_dict[item])
print(total)

127


KeyError: "['TotalBsmtSF', 'BsmtFinType1', 'BsmtFinType2'] not in index"

In [ ]:
#np.unique(data_in[data_in['BsmtFinType1']=='NA']['BsmtFinSF1'])
np.sum(data_in['TotalBsmtSF'] == data_in['BsmtUnfSF'] + data_in['BsmtFinSF1'] + data_in['BsmtFinSF2'])

1094

In [36]:
type(data_in['MoSold'].loc[0])

numpy.int64